## Step 1: Dependencies

Check that all necessary libraries are added to your local environment:

In [1]:
import warnings
warnings.filterwarnings('ignore')

import tensorflow as tf
#pip install keras-bert

import os
import warnings
import sys
import codecs
import numpy as np
import argparse
import json
import pickle
import pandas as pd

from util import read_passages, evaluate, make_folds, clean_words, test_f1, to_BIO, from_BIO, from_BIO_ind, arg2param

import tensorflow as tf
#config = tf.ConfigProto()
config = tf.compat.v1.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 1.0
config.gpu_options.allow_growth = True  # dynamically grow the memory used on the GPU
#sess = tf.Session(config=config)
sess = tf.compat.v1.Session()
#import keras.backend as K
from tensorflow.compat.v1.keras import backend as K
K.set_session(sess)
from keras.activations import softmax
from keras.regularizers import l2
from keras.models import Model, model_from_json
from keras.layers import Input, LSTM, Dense, Dropout, TimeDistributed, Bidirectional
from keras.callbacks import EarlyStopping,LearningRateScheduler, ModelCheckpoint
from keras.optimizers import Adam, RMSprop, SGD

from crf import CRF
from attention import TensorAttention
from custom_layers import HigherOrderTimeDistributedDense
from generator import BertDiscourseGenerator

from keras_bert import load_trained_model_from_checkpoint, Tokenizer

import discourse_tagger_generator_bert
# import bert
# from transformers import AutoTokenizer, AutoModel 
#!pip install transformers
# from transformers import pipeline

#!pip install --upgrade tensorflow
#CHECK TENSORFLOW VERSION - Should be 2.3.0
tf.version.VERSION

'2.3.0'

In [2]:
articles = os.listdir('../data/testfiles/sampleArticles')

areas = ['BIOC']

for article in articles:
    i = 0
#     path = ['data', 'open_cc', 'json', article]
    with open(os.path.join('../data', 'testfiles', 'sampleArticles', article)) as file:        
        full_text = json.load(file)
        try:
            highlights = full_text['author_highlights']
            author_highlights = []
            for item in highlights:
                item = item['sentence'].strip()
                if not item.endswith('.'):
                    item += '.'  # ensures each is a "sentence"
                author_highlights.append(item)
            

            if 'BIOC' in full_text['metadata']['subjareas']:
                abstract = full_text['abstract']
                body_text = full_text['body_text']
                text = [t['sentence'] for t in body_text if t['title'] == 'Discussion']

        except KeyError:
            pass
    i += 1
    if i > 0:
        break

In [3]:
author_highlights

['We investigate the pro-apoptotic effects of PETIC in human brain GBM 8401 cells.',
 'PEITC decreases the cell viability of GBM 8401 cells.',
 'PEITC induces significantly sub-G1 phase in GBM 8401 cells.',
 'PEITC induces apoptosis through multiple pathways in GBM 8401 cells.',
 'We postulate the signaling pathways of PEITC for human brain glioblastoma cells.']

In [4]:
text[:3]

['On the contrary, TMZ did not increase ROS production, Ca2+ release, but is with involvement of mitochondria changes in GBM 8401 cells (Fig. 3).',
 'The central effectors of apoptosis are caspases.',
 'ER stress markers include GADD153, GRP 78, IRE-1α, XBP-1 (Gotoh et al., 2004; Kim et al., 2007; Lovat et al., 2003; Oyadomari et al., 2002; Tajiri et al., 2004).']

In [5]:
with open('../data/testFiles/S0197018615000054.txt', 'w', encoding="utf-8") as f:
    for item in text:
        f.write("%s\n" % item)

# Tag Example File:

Pass your test file in the command below with the same parameters as the trained model. 

In [6]:
%run discourse_tagger_generator_bert.py --repfile ../models/scibert_scivocab_uncased --test_file ../data/testFiles/S0197018615000054.txt --att_context LSTM_clause --bidirectional --save --maxseqlen 40 --maxclauselen 60 --batch_size 20

att=False_cont=LSTM_clause_lstm=False_bi=True_crf=False
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. W

# Load Predictions & Article into Dataframes

In [7]:
dfPredictions = pd.read_csv('predictions/S0197018615000054att=False_cont=LSTM_clause_lstm=False_bi=True_crf=False.out',names=['DST'])
dfArticle = pd.DataFrame(text, columns=['Sentence'])
dfFinal = pd.concat([dfArticle, dfPredictions], axis=1)
dfFinal.head(8)

# - To see non-truncated values, run: pd.set_option('display.max_colwidth', -1)

,Sentence,DST
0,"On the contrary, TMZ did not increase ROS prod...",result
1,The central effectors of apoptosis are caspases.,result
2,"ER stress markers include GADD153, GRP 78, IRE...",result
3,The effects of PEITC on human glioma cells hav...,result
4,"In our study, PEITC promoted the protein expre...",result
5,PEITC induced cytotoxic effects on GBM 8401 hu...,result
6,The intrinsic (mitochondrial) pathways result ...,result
7,"According to these observations above, we sugg...",implication


# Filter Out Implication Clauses

In [8]:
dfImplications = dfFinal[dfFinal['DST'] == 'implication']
dfImplications

,Sentence,DST
7,"According to these observations above, we sugg...",implication
14,"Thus, TMZ has various effects on caspases or i...",implication


# Compare to Research Highlights

In [9]:
author_highlights

['We investigate the pro-apoptotic effects of PETIC in human brain GBM 8401 cells.',
 'PEITC decreases the cell viability of GBM 8401 cells.',
 'PEITC induces significantly sub-G1 phase in GBM 8401 cells.',
 'PEITC induces apoptosis through multiple pathways in GBM 8401 cells.',
 'We postulate the signaling pathways of PEITC for human brain glioblastoma cells.']

In [10]:
dfImplications

,Sentence,DST
7,"According to these observations above, we sugg...",implication
14,"Thus, TMZ has various effects on caspases or i...",implication


## Author Highlights:

* We investigate the pro-apoptotic effects of PETIC in human brain GBM 8401 cells.
* PEITC decreases the cell viability of GBM 8401 cells.
* PEITC induces significantly sub-G1 phase in GBM 8401 cells.
* PEITC induces apoptosis through multiple pathways in GBM 8401 cells.
* We postulate the signaling pathways of PEITC for human brain glioblastoma cells.


## Implication Clause(s):
* According to these observations above, we suggest that PEITC induces apoptosis also through intrinsic (mitochondrial) pathways as well as extrinsic (death receptor) ones.
* Thus, TMZ has various effects on caspases or induces apoptosis through different pathways in different glioblastoma cell lines, and personalized treatments for glioblastoma are necessary.

In [13]:
# - Optional save file for further analysis / comparison to research highlights
#dfFinal.to_csv(r'../../exampleArticle.csv', index = False)